In [295]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from scipy import stats
from helper import *
from scipy.stats.mstats import gmean
#Importation of all the packages
import datetime
import math
import json
import zipfile  
import ssl
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
import plotly.express as px

#To dowload data
import requests
import io
import gzip

#To create the mapchart
import iso3166
import plotly
from iso3166 import countries
import plotly.graph_objects as go

In [296]:
pageview_df = pd.read_csv("page_views_covid_related.csv.gz")
population_df = pd.read_csv("Population_countries.csv")

In [297]:

def get_pageviews_df(raw_pageview_df: pd.DataFrame, population_df: pd.DataFrame, country_dict: dict, start: str, end: str):
    '''
    Function to get the different pageviews dataset
    
    Inputs : 

        - raw_pageview_df : raw dataset from csv
        - population_df : raw population dataset from csv
        - country_dict : countries we are interested in with language code
        - start : start date (yyyy-mm-dd)
        - end : end date (yyyy-mm-dd)
        
    Output : df_pageviews, df_pageviews_cumul, df_pageviews100k, df_pageviews_cumul100k
    '''
    #inv_country_dict = {v: k for k, v in country_dict.items()}
    pageview_df_imp_country = raw_pageview_df[["date"] + list(country_dict.values())].set_index('date')
    df_pageviews = pageview_df_imp_country.loc[pageview_df_imp_country.index < end]
    df_pageviews = df_pageviews.loc[df_pageviews.index >= start]
    df_pageviews = df_pageviews.interpolate(method ='linear', limit_direction ='forward') 
    df_pageviews = df_pageviews.fillna(0)

    df_pageviews_cumul = df_pageviews.cumsum()

    COUNTRY_OWN_LANG_POP = {"Italy" : "it", "Russian Federation": "ru", "China": "zh", "Albania": "sq", 
    "Bangladesh": "bn", "Botswana": "tn", "Cambodia": "km", "Croatia": "hr", "Greece": "el", "Sweden": "sv", "Finland": "fi", "Norway": "no",
    "Malaysia": "ms", "Israel": "he", "Lithuania": "lt", "Serbia": "sr", "Slovak Republic": "sk", "Slovenia": "sl", "Turkiye": "tr",
    "Vietnam": "vi", "Bulgaria": "bg", "Czechia": "cs", "Denmark": "da", "Georgia": "ka", "Germany": "de", 
    "Hungary": "hu", "Iceland": "is", "Japan": "ja", "Kazakhstan": "kk", "Korea, Rep.": "ko", "Kyrgyz Republic": 'ky', "Netherlands": "nl", "Poland": "pl", 
    "Romania": "ro", "Tajikistan": "tg", "Thailand": "th", "Azerbaijan": "az", "Mongolia": "mn"}

    population_df = population_df[["Country Name", "2020"]]
    population_df = population_df.set_index("Country Name")
    population_df = population_df.transpose()
    population_df = population_df[list(COUNTRY_OWN_LANG_POP.keys())]
    population_df = population_df.rename(columns= COUNTRY_OWN_LANG_POP)

    df_pageviews_cumul100k = df_pageviews_cumul/population_df.values * 100000
    df_pageviews100k = df_pageviews/population_df.values * 100000
    return df_pageviews, df_pageviews_cumul, df_pageviews100k, df_pageviews_cumul100k

In [298]:

def get_cases_deaths_df(population_df: pd.DataFrame, country_dict: dict, start: str, end: str):
    '''
    Function to get the different COVID cases and deaths dataset
    
    Inputs : 

        - population_df : raw population dataset from csv
        - country_dict : countries we are interested in with language code
        - start : start date (yyyy-mm-dd)
        - end : end date (yyyy-mm-dd)

    Output : deaths, cases, deaths_cumul, cases_cumul, deaths100k, deaths100k_cumul, cases100k, cases100k_cumul
    '''
    ssl._create_default_https_context = ssl._create_unverified_context
    death_url = "https://github.com/owid/covid-19-data/blob/master/public/data/jhu/new_deaths.csv?raw=true" # Make sure the url is the raw version of the file on GitHub
    cases_url = "https://github.com/owid/covid-19-data/blob/master/public/data/jhu/new_cases.csv?raw=true"

    # Reading the downloaded content and turning it into a pandas dataframe
    deaths = pd.read_csv(death_url,index_col=0)
    cases = pd.read_csv(cases_url,index_col=0)
    deaths= deaths.interpolate(method ='linear', limit_direction ='forward') 
    cases = cases.interpolate(method ='linear', limit_direction ='forward') 
    deaths = deaths.fillna(0)
    cases = cases.fillna(0)
    #Keep only values between start and end
    deaths = deaths[deaths.index < end]
    cases = cases[cases.index < end] 
    deaths = deaths[deaths.index >= start]
    cases = cases[cases.index >= start]

    deaths = deaths.rename(columns= country_dict)[country_dict.values()]
    cases = cases.rename(columns= country_dict)[country_dict.values()]

    deaths_cumul = deaths.cumsum()
    cases_cumul = cases.cumsum()
    
    COUNTRY_OWN_LANG_POP = {"Italy" : "it", "Russian Federation": "ru", "China": "zh", "Albania": "sq", 
    "Bangladesh": "bn", "Botswana": "tn", "Cambodia": "km", "Croatia": "hr", "Greece": "el", "Sweden": "sv", "Finland": "fi", "Norway": "no",
    "Malaysia": "ms", "Israel": "he", "Lithuania": "lt", "Serbia": "sr", "Slovak Republic": "sk", "Slovenia": "sl", "Turkiye": "tr",
    "Vietnam": "vi", "Bulgaria": "bg", "Czechia": "cs", "Denmark": "da", "Georgia": "ka", "Germany": "de", 
    "Hungary": "hu", "Iceland": "is", "Japan": "ja", "Kazakhstan": "kk", "Korea, Rep.": "ko", "Kyrgyz Republic": 'ky', "Netherlands": "nl", "Poland": "pl", 
    "Romania": "ro", "Tajikistan": "tg", "Thailand": "th", "Azerbaijan": "az", "Mongolia": "mn"}

    population_df = population_df[["Country Name", "2020"]]
    population_df = population_df.set_index("Country Name")
    population_df = population_df.transpose()
    population_df = population_df[list(COUNTRY_OWN_LANG_POP.keys())]
    population_df = population_df.rename(columns= COUNTRY_OWN_LANG_POP)

    deaths100k = deaths/population_df.values * 100000
    deaths100k_cumul = deaths_cumul/population_df.values * 100000
    cases100k = cases/population_df.values * 100000
    cases100k_cumul = cases_cumul/population_df.values * 100000
    return deaths, cases, deaths_cumul, cases_cumul, deaths100k, deaths100k_cumul, cases100k, cases100k_cumul
    

### Cumul pageviews, cases and deaths during 1st wave beteween 2020-01-22 and 2020-05-22.

In [299]:
_, _, _, df_pageviews_cumul100k = get_pageviews_df(pageview_df, population_df, get_country_dict('original'), '2020-01-22', '2020-05-22')
_, _, _, _, _, deaths100k_cumul, _, cases100k_cumul = get_cases_deaths_df(population_df, get_country_dict('original'), '2020-01-22', '2020-05-22')

In [300]:
data = pd.DataFrame()
data['y'] = np.log(df_pageviews_cumul100k.max().transpose().replace(0, np.nan))
data['x2'] = np.log(cases100k_cumul.max().transpose().replace(0, np.nan))
data['x'] = np.log(deaths100k_cumul.max().transpose().replace(0, np.nan))
data = data.dropna()
data = data.rename(index= {v: k for k, v in get_country_dict('original').items()}).reset_index().rename(columns = {'index': 'Country'})

#We apply a linear regression using as estimates OLS
model = smf.ols('y ~ x', data=data).fit()

results = model.summary()

#Show the results of the linear regression
print("Our model summary is:")
results

Our model summary is:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.497
Method:                 Least Squares   F-statistic:                     34.63
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           1.35e-06
Time:                        10:10:52   Log-Likelihood:                -59.005
No. Observations:                  35   AIC:                             122.0
Df Residuals:                      33   BIC:                             125.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.4383      0.237     31.445      0.000       6.957       7.920
x              0.7979      0.136      5.885      0.000       0.522       1.074
==============================================================================
Omnibus:                        6.063   Durbin-Watson:                   2.411
Prob(Omnibus):                  0.048   Jarque-Bera (JB):                4.561
Skew:                          -0.810   Prob(JB):                        0.102
Kurtosis:                       3.711   Cond. No.                         1.88
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [301]:
fig = px.scatter(data, x='x', y="y", hover_name= 'Country',trendline="ols", title='Relationship between Log of cumulative deaths and Log of cumulative pageviews')

fig.update_layout(margin=dict(l=20, r=20, t=100, b=0))  
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.13,showarrow=False,text = "    The x axis shows the log of the cumulative deaths for 100k of population.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.09,showarrow=False,text = "    The y axis shows the log of the cumulative pageviews for 100k of population.")
fig.show()


In [302]:
model = smf.ols('y ~ x2', data=data).fit()

results = model.summary()

#Show the results of the linear regression
print("Our model summary is:")
results

Our model summary is:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     19.65
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           9.71e-05
Time:                        10:10:52   Log-Likelihood:                -63.387
No. Observations:                  35   AIC:                             130.8
Df Residuals:                      33   BIC:                             133.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.4021      0.814      5.410      0.000       2.747       6.057
x2             0.8640      0.195      4.433      0.000       0.467       1.261
==============================================================================
Omnibus:                        7.102   Durbin-Watson:                   2.588
Prob(Omnibus):                  0.029   Jarque-Bera (JB):                5.666
Skew:                          -0.921   Prob(JB):                       0.0588
Kurtosis:                       3.703   Cond. No.                         13.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [303]:
fig = px.scatter(data, x='x2', y="y", hover_name= 'Country',trendline="ols", title='Relationship between Log of cumulative cases and Log of cumulative pageviews')

fig.update_layout(margin=dict(l=20, r=20, t=100, b=0))  
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.13,showarrow=False,text = "    The x axis shows the log of the cumulative cases for 100k of population.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.09,showarrow=False,text = "    The y axis shows the log of the cumulative pageviews for 100k of population.")
fig.show()


### Cumul pageviews, cases and deaths during 1st wave beteween 2022-02-01 and 2022-07-01.

In [304]:
_, _, _, df_pageviews_cumul100k = get_pageviews_df(pageview_df, population_df, get_country_dict('original'), '2022-02-01', '2022-07-01')
_, _, _, _, _, deaths100k_cumul, _, cases100k_cumul = get_cases_deaths_df(population_df, get_country_dict('original'), '2022-02-01', '2022-07-01')

In [305]:
data = pd.DataFrame()
data['y'] = np.log(df_pageviews_cumul100k.max().transpose().replace(0, np.nan))
data['x2'] = np.log(cases100k_cumul.max().transpose().replace(0, np.nan))
data['x'] = np.log(deaths100k_cumul.max().transpose().replace(0, np.nan))
data = data.dropna()
data = data.rename(index= {v: k for k, v in get_country_dict('original').items()}).reset_index().rename(columns = {'index': 'Country'})

#We apply a linear regression using as estimates OLS
model = smf.ols('y ~ x', data=data).fit()

results = model.summary()

#Show the results of the linear regression
print("Our model summary is:")
results

Our model summary is:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     13.04
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           0.000947
Time:                        10:10:54   Log-Likelihood:                -51.947
No. Observations:                  37   AIC:                             107.9
Df Residuals:                      35   BIC:                             111.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      4.4250      0.314     14.081      0.000       3.787       5.063
x              0.3696      0.102      3.611      0.001       0.162       0.577
==============================================================================
Omnibus:                        3.640   Durbin-Watson:                   2.142
Prob(Omnibus):                  0.162   Jarque-Bera (JB):                1.596
Skew:                           0.010   Prob(JB):                        0.450
Kurtosis:                       1.983   Cond. No.                         6.25
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [306]:
fig = px.scatter(data, x='x', y="y", hover_name= 'Country',trendline="ols", title='Relationship between Log of cumulative deaths and Log of cumulative pageviews')

fig.update_layout(margin=dict(l=20, r=20, t=100, b=0))  
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.13,showarrow=False,text = "    The x axis shows the log of the cumulative deaths for 100k of population.")
fig.add_annotation(xref='paper',yref='paper',x=0, y=1.09,showarrow=False,text = "    The y axis shows the log of the cumulative pageviews for 100k of population.")
fig.show()

In [307]:
model = smf.ols('y ~ x2', data=data).fit()

results = model.summary()

#Show the results of the linear regression
print("Our model summary is:")
results

Our model summary is:


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.405
Model:                            OLS   Adj. R-squared:                  0.388
Method:                 Least Squares   F-statistic:                     23.86
Date:                Wed, 21 Dec 2022   Prob (F-statistic):           2.27e-05
Time:                        10:10:54   Log-Likelihood:                -48.188
No. Observations:                  37   AIC:                             100.4
Df Residuals:                      35   BIC:                             103.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.8711      0.735      2.544      0.016       0.378       3.364
x2             0.4225      0.086      4.885      0.000       0.247       0.598
==============================================================================
Omnibus:                        1.090   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.580   Jarque-Bera (JB):                0.925
Skew:                          -0.128   Prob(JB):                        0.630
Kurtosis:                       2.269   Cond. No.                         42.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [308]:
from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

_, _, df_pageviews_100k, df_pageviews_cumul100k = get_pageviews_df(pageview_df, population_df, get_country_dict('original'), '2020-01-22', '2022-07-01')
_, _, _, _, deaths100k, deaths100k_cumul, cases100k, cases100k_cumul = get_cases_deaths_df(population_df, get_country_dict('original'), '2020-01-22', '2022-07-01')

In [309]:
data = pd.DataFrame()
deaths100k = deaths100k.rename(columns= {v: k for k, v in get_country_dict('original').items()}).reset_index().rename(columns = {'index': 'Country'})
deaths100k = deaths100k.drop('date', axis= 1)
df_pageviews_100k = df_pageviews_100k.rename(columns= {v: k for k, v in get_country_dict('original').items()}).reset_index().rename(columns = {'index': 'Country'})
df_pageviews_100k = df_pageviews_100k.drop('date', axis= 1)

df_pageviews_100k_rollingcumul = df_pageviews_100k.rolling(60, min_periods=1).sum()
df_pageviews_100k_rollingcumul = df_pageviews_100k_rollingcumul.loc[df_pageviews_100k_rollingcumul.index % 60 == 59]
df_pageviews_100k_rollingcumul = df_pageviews_100k_rollingcumul.reset_index().drop('index', axis= 1)

deaths100k_rollingcumul = deaths100k.rolling(60, min_periods=1).sum()
deaths100k_rollingcumul = deaths100k_rollingcumul.loc[deaths100k_rollingcumul.index % 60 == 59]
deaths100k_rollingcumul = deaths100k_rollingcumul.reset_index().drop('index', axis= 1)

In [324]:
data = pd.DataFrame()
data_tmp = pd.DataFrame()
for i in df_pageviews_100k_rollingcumul.index.values:
    if i == 0:
        data['Log of Pageviews'] = np.log(df_pageviews_100k_rollingcumul.transpose()[i] + 0.01)
        data['Log of Deaths'] = np.log(deaths100k_rollingcumul.transpose()[i] + 0.01)
        data['i'] = i
    else:
        data_tmp ['Log of Pageviews'] = np.log(df_pageviews_100k_rollingcumul.transpose()[i] + 0.01)
        data_tmp ['Log of Deaths'] = np.log(deaths100k_rollingcumul.transpose()[i] + 0.01)
        data_tmp ['i'] = i
        data = pd.concat([data, data_tmp], axis= 0)

data = data.reset_index().rename(columns= {'index' : 'country'}).dropna()
data['Date'] = data['i'].apply(lambda x:pd.to_datetime('2020-01-22') + datetime.timedelta(days= x * 59 + 59))
data['Date'] = data['Date'].astype(str)
data


,country,Log of Pageviews,Log of Deaths,i,Date
0,Italy,8.893485,2.095085,0,2020-03-21
1,Russia,8.812605,-4.538064,0,2020-03-21
2,China,6.335957,-1.428455,0,2020-03-21
3,Albania,4.759460,-2.519797,0,2020-03-21
4,Bangladesh,2.609923,-4.490556,0,2020-03-21
...,...,...,...,...,...
527,Romania,3.854435,1.914946,13,2022-04-27
528,Tajikistan,1.268032,-4.605170,13,2022-04-27
529,Thailand,2.946923,2.087287,13,2022-04-27
530,Azerbaijan,4.123657,0.181428,13,2022-04-27


In [339]:

fig = px.scatter(data, x="Log of Deaths", y="Log of Pageviews", animation_frame="Date", hover_name="country", trendline="ols", trendline_color_override= 'black') #animation_group='i'
fig.update_traces(marker_size=10, marker_color= 'orange')
fig.update_layout(height=600, width= 1200)
fig.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000
fig.show()


results = px.get_trendline_results(fig)
for i in range(14):
    results1 = results.iloc[i]["px_fit_results"].params
    results2 = results.iloc[i]["px_fit_results"].pvalues
    date = results.iloc[i]["Date"]
    print('Regression on 2 months cumulative log-pageviews and log-deaths until {}'.format(date))
    print('Intercept= {:.2}, Coefficient= {:.2}'.format(results1[0], results1[1]))
    print('Pvalues of intercept= {:.2}, Pvalues of coefficient= {:.2} \n'.format(results2[0], results2[1]))

Regression on 2 months cumulative log-pageviews and log-deaths until 2020-03-21
Intercept= 8.9, Coefficient= 0.83
Pvalues of intercept= 5.8e-09, Pvalues of coefficient= 0.015 

Regression on 2 months cumulative log-pageviews and log-deaths until 2020-05-19
Intercept= 7.1, Coefficient= 0.65
Pvalues of intercept= 6.8e-27, Pvalues of coefficient= 1e-06 

Regression on 2 months cumulative log-pageviews and log-deaths until 2020-07-17
Intercept= 6.2, Coefficient= 0.24
Pvalues of intercept= 1.9e-22, Pvalues of coefficient= 0.04 

Regression on 2 months cumulative log-pageviews and log-deaths until 2020-09-14
Intercept= 5.8, Coefficient= 0.21
Pvalues of intercept= 1e-21, Pvalues of coefficient= 0.098 

Regression on 2 months cumulative log-pageviews and log-deaths until 2020-11-12
Intercept= 5.5, Coefficient= 0.46
Pvalues of intercept= 2.5e-24, Pvalues of coefficient= 3.8e-06 

Regression on 2 months cumulative log-pageviews and log-deaths until 2021-01-10
Intercept= 5.2, Coefficient= 0.39
Pv